## Инициализация проекта

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings


# Исследование данных

### Вывод базовой информации

In [36]:
data = pd.read_csv('../data/trip_duration_task.csv')

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 729322 non-null  object 
 1   vendor_id          727135 non-null  float64
 2   pickup_datetime    729322 non-null  object 
 3   dropoff_datetime   729322 non-null  object 
 4   passenger_count    729322 non-null  int64  
 5   pickup_longitude   729322 non-null  float64
 6   pickup_latitude    727475 non-null  float64
 7   dropoff_longitude  729322 non-null  float64
 8   dropoff_latitude   729322 non-null  float64
 9   trip_duration      729322 non-null  int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 55.6+ MB


In [38]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,id1080784,2.0,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,400
1,id0889885,1.0,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,1100
2,id0857912,2.0,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,1635
3,id3744273,2.0,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,1141
4,id0232939,1.0,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,848


### Обработка столбцов

Исходный датасет содержит несколько столбцов, которые не содержат информации, имеющей свзять с продолжительностью поездки: id

In [39]:
data.drop(columns=['id', 'passenger_count'], inplace=True)
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2.0,2016-02-29 16:40:21,2016-02-29 16:47:01,-73.953918,40.778873,-73.963875,40.771164,400
1,1.0,2016-03-11 23:35:37,2016-03-11 23:53:57,-73.988312,40.731743,-73.994751,40.694931,1100
2,2.0,2016-02-21 17:59:33,2016-02-21 18:26:48,-73.997314,40.721458,-73.948029,40.774918,1635
3,2.0,2016-01-05 09:44:31,2016-01-05 10:03:32,-73.961670,40.759720,-73.956779,40.780628,1141
4,1.0,2016-02-17 06:42:23,2016-02-17 06:56:31,-74.017120,40.708469,-73.988182,40.740631,848


### Исключительный случай

Датасет содержит столбец содержащий информацию о времени начала поездки, что вплоне логично для задачи расчета продолжительности поездки, а также содержит столбец времени окончания поездки, что нелогично для данной задачи. Поэтому я решил удалить столбец времени окончания поездки, чтобы симулировать настояющую бизнес-задачу

# Обогощение данных

In [40]:
data = data.dropna(subset=['pickup_latitude'])

In [41]:
from geopy.distance import geodesic

data['distance'] = data.apply(
    lambda row: geodesic((row['pickup_latitude'], row['pickup_longitude']),
                          (row['dropoff_latitude'], row['dropoff_longitude'])).meters, axis=1
)
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,distance
0,2.0,2016-02-29 16:40:21,2016-02-29 16:47:01,-73.953918,40.778873,-73.963875,40.771164,400,1199.769914
1,1.0,2016-03-11 23:35:37,2016-03-11 23:53:57,-73.988312,40.731743,-73.994751,40.694931,1100,4123.945168
2,2.0,2016-02-21 17:59:33,2016-02-21 18:26:48,-73.997314,40.721458,-73.948029,40.774918,1635,7250.435807
3,2.0,2016-01-05 09:44:31,2016-01-05 10:03:32,-73.961670,40.759720,-73.956779,40.780628,1141,2358.286547
4,1.0,2016-02-17 06:42:23,2016-02-17 06:56:31,-74.017120,40.708469,-73.988182,40.740631,848,4328.155268
